In [2]:

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


data =pd.read_csv("C:/Users/habee/OneDrive/Desktop/Dataset .csv")

print(data.head())


columns_to_keep = ['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating', 'Votes']
data = data[columns_to_keep]


data.fillna('', inplace=True)


scaler = StandardScaler()
data[['Price range', 'Aggregate rating', 'Votes']] = scaler.fit_transform(data[['Price range', 'Aggregate rating', 'Votes']])


tfidf = TfidfVectorizer(stop_words='english')
cuisine_matrix = tfidf.fit_transform(data['Cuisines'])


features = np.hstack((cuisine_matrix.toarray(), data[['Price range', 'Aggregate rating', 'Votes']].values))


similarity_matrix = cosine_similarity(features)


def recommend_restaurants(user_preferences, top_n=5):
    """
    Recommend restaurants based on user preferences.
    
    Parameters:
    user_preferences (dict): A dictionary with keys 'Cuisines', 'Price range', 'Aggregate rating', and 'Votes'
    top_n (int): Number of recommendations to return
    
    Returns:
    DataFrame: Recommended restaurants
    """
    
    user_data = pd.DataFrame([user_preferences])
    
   
    user_cuisine_vector = tfidf.transform(user_data['Cuisines'])
    
    
    user_data[['Price range', 'Aggregate rating', 'Votes']] = scaler.transform(
        user_data[['Price range', 'Aggregate rating', 'Votes']]
    )
    
   
    user_features = np.hstack((user_cuisine_vector.toarray(), user_data[['Price range', 'Aggregate rating', 'Votes']].values))
    
    
    similarities = cosine_similarity(user_features, features)
    
   
    recommended_indices = np.argsort(similarities[0])[::-1][:top_n]
    
    return data.iloc[recommended_indices][['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating', 'Votes']]


user_preferences = {
    'Cuisines': 'Italian, Pizza',
    'Price range': 2,  # 1 to 4 (Low to High)
    'Aggregate rating': 4.0,  # User prefers high-rated restaurants
    'Votes': 500  # User prefers popular restaurants with high votes
}


recommendations = recommend_restaurants(user_preferences)
print("Recommended Restaurants:\n", recommendations)


   Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Mandaluyong City   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   

                                     Locality  \
0   Century City Mall, Poblacion, Makati City   
1  Little Tokyo, Legaspi Village, Makati City   
2  Edsa Shangri-La, Ortigas, Mandaluyong City   
3      SM 